# HW 5
## Linear Regression Error

Given noisy target y = **w**<sup>*T</sup>x + &epsilon; &SuchThat; x &isin; &reals;<sup>d</sup> (with x<sub>0</sub> = 1), y &isin; &reals;, w* unknown vector, &epsilon; noise term with zero mean and &sigma;<sup>2</sup> variance; data set &Dscr; = {(x1, y1),...,(xN, yN)} and the equation:


&Eopf;<sub>&Dscr;</sub>[E<sub>in</sub>(w<sub>lin</sub>)] = &sigma;<sup>2</sup>(1 - (d+1)/N)


Let the LH be &tau;, then:

&tau;/(&sigma;<sup>2</sup>) = 1 - (d+1)/N

&rArr; (d+1)/N = 1-&tau;/(&sigma;<sup>2</sup>)

&rArr; (d+1)/(1-&tau;/(&sigma;<sup>2</sup>)) = N



For &sigma; = 0.1, d = 8, E<sub>in</sub> &gt; 0.008, want smallest N.

&rArr; 9/(1 - 0.008/(0.1<sup>2</sup>)) < N

&rArr; 45 < N


Thus the smallest given N that satisfies this is **100**



## Nonlinear Transforms

**Given :** &phi;(1, x1, x2) = (1, x1<sup>2</sup>, x2<sup>2</sup>)

**Want :** weights giving a hyperbolic decision boundary



Since the general equation of a hyperbola is x<sup>2</sup>/a<sup>2</sup> - y<sup>2</sup>/b<sup>2</sup> = 1 and we want h(x) to be negative when x1<sup>2</sup> is large, the correct weights should be **w1 < 0, w2 > 0**.

**Given :** &phi;<sub>4</sub>: x &rarr; (1, x1, x1<sup>2</sup>, x1*x2, x2<sup>2</sup>, x1<sup>3</sup>, x1<sup>2</sup>*x2, x1*x2<sup>2</sup>, x2<sup>3</sup>, x1<sup>4</sup>, x1<sup>3</sup>*x2, x1<sup>2</sup>*x2<sup>2</sup>, x1*x2<sup>3</sup>, x2<sup>4</sup>), d = 13.

**Want :** VC dimension


Since VC dimension for a linear model = d + 1, d<sub>VC</sub> = 14, thus the lowest number given not smaller than the VC dimension is **15**.